In [1]:
#import libraries.
import numpy as np
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
from tensorflow.python.keras.utils.data_utils import Sequence
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
import keras
from keras import regularizers
import sklearn
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import sounddevice as sd
import tensorflow as tf
from mir_eval import separation 
from pystoi.stoi import stoi 
import h5py
from keras.callbacks import LearningRateScheduler
from keras.layers.normalization import BatchNormalization
from keras.layers import Activation
import math
from sklearn.utils import shuffle

Using TensorFlow backend.


In [2]:
#import libraries.
# import matplotlib.pyplot as plt
# from tabulate import tabulate
import time
import os
import librosa
from librosa.core import stft, istft
import time
import pickle
from keras import backend as K
from tensorflow.keras.models import load_model
# from keras.models import load_model

In [3]:
def reconstruct(wave,angle):
    recon1 = wave*np.cos(angle)+wave*np.sin(angle)*1j
#     recon = np.sqrt(np.power(10, wave))
#     recon1 = recon*np.cos(angle)+recon*np.sin(angle)*1j
    recon = librosa.core.istft((recon1.T), hop_length=160, win_length=400, window='hann')
    return recon

In [4]:
estimator_A = load_model('Models/Two_stage/trained_model12.h5')

Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [7]:
# FF
h5f = h5py.File('TIMIT/Organized/concatenated/Test/Mixed_FF_test4.hdf5','r')
test_input = h5f['mixed_FF_test4'][:]
h5f.close()
h5f = h5py.File('TIMIT/Organized/concatenated/Test/clean_FF_test4.hdf5','r')
test_target = h5f['clean_FF_test4'][:]
h5f.close()
h5f = h5py.File('TIMIT/Organized/concatenated/Test/clean_FF_test_phase4.hdf5','r')
angle_clean = h5f['clean_FF_test-phase4'][:]
h5f.close()
h5f = h5py.File('TIMIT/Organized/concatenated/Test/Mixed_FF_test_phase4.hdf5','r')
angle_mixed = h5f['mixed_FF_test_phase4'][::]
h5f.close()

In [48]:
# MM
h5f = h5py.File('TIMIT/Organized/concatenated/Test/Mixed_MM_test4.hdf5','r')
test_input = h5f['mixed_MM_test4'][:]
h5f.close()
h5f = h5py.File('TIMIT/Organized/concatenated/Test/clean_MM_test4.hdf5','r')
test_target = h5f['clean_MM_test4'][:]
h5f.close()
h5f = h5py.File('TIMIT/Organized/concatenated/Test/clean_MM_test_phase4.hdf5','r')
angle_clean = h5f['clean_MM_test_phase4'][:]
h5f.close()
h5f = h5py.File('TIMIT/Organized/concatenated/Test/Mixed_MM_test_phase4.hdf5','r')
angle_mixed = h5f['mixed_MM_test_phase4'][::]
h5f.close()

In [80]:
# FM
h5f = h5py.File('TIMIT/Organized/concatenated/Test/Mixed_FM_test4.hdf5','r')
test_input = h5f['mixed_FM_test4'][:]
h5f.close()
h5f = h5py.File('TIMIT/Organized/concatenated/Test/clean_FM_test4.hdf5','r')
test_target = h5f['clean_FM_test4'][:]
h5f.close()
h5f = h5py.File('TIMIT/Organized/concatenated/Test/clean_FM_test_phase4.hdf5','r')
angle_clean = h5f['clean_FM_test_phase4'][:]
h5f.close()
h5f = h5py.File('TIMIT/Organized/concatenated/Test/Mixed_FM_test_phase4.hdf5','r')
angle_mixed = h5f['mixed_FM_test_phase4'][::]
h5f.close()

In [81]:
test_input = sklearn.preprocessing.normalize(test_input, norm='l2', axis=1, copy=True, return_norm=False)

# Data1 = sklearn.preprocessing.normalize(Data1, norm='l2', axis=1, copy=True, return_norm=False)

In [82]:
test_target = sklearn.preprocessing.normalize(test_target, norm='l2', axis=1, copy=True, return_norm=False)

In [83]:
print(test_input.shape)
print(test_target.shape)
print(angle_clean.shape)
print(angle_mixed.shape)

(18634, 257)
(18634, 514)
(18634, 514)
(18634, 257)


In [84]:
fft_samples = 257

In [85]:
prediction1 = estimator_A.predict(test_input)

In [115]:
speaker1 = np.multiply(test_input, prediction1[:,0:fft_samples])
speaker2 = np.multiply(test_input, prediction1[:,fft_samples:])

In [102]:
# prediction[1000]

In [122]:
s1 = reconstruct(speaker1, angle_mixed)
s2 = reconstruct(speaker2, angle_mixed)

In [119]:
clean_recon1 = reconstruct(test_target[:,0:fft_samples], angle_clean[:,0:fft_samples])
clean_recon2 = reconstruct(test_target[:,fft_samples:], angle_clean[:,fft_samples:])

In [120]:
mixed_recon = reconstruct(test_input[:], angle_mixed)

In [123]:
st_recon1 = stoi(s1, clean_recon1, 16000, extended=False)
st_recon2 = stoi(s2, clean_recon2, 16000, extended=False)

In [124]:
st_mixed = stoi(mixed_recon, clean_recon, 16000, extended=False)

In [125]:
print('STOI_recon1=', st_recon1)
print('STOI_recon2=', st_recon2)
print('STOI_mixed=', st_mixed)

STOI_recon1= 0.6571472273996947
STOI_recon2= 0.6095785571954762
STOI_mixed= 0.5201365922978889


In [126]:
sdr_recon1, sir_recon1, sar_recon1, perm_recon1 = separation.bss_eval_sources(s1, clean_recon1, compute_permutation=True)
sdr_recon2, sir_recon2, sar_recon2, perm_recon2 = separation.bss_eval_sources(s2, clean_recon2, compute_permutation=True)

In [127]:
sdr_mixed, sir_mixed, sar_mixed, perm_mixed = separation.bss_eval_sources(mixed_recon, clean_recon, compute_permutation=True)

In [128]:
print('SDR_recon1=', sdr_recon1)
print('SDR_recon2=', sdr_recon2)
print('SDR_mixed=', sdr_mixed)

SDR_recon1= [1.25020044]
SDR_recon2= [2.47262778]
SDR_mixed= [-0.44207504]


In [134]:
# start = int(len(s1)/20)
# stop = int(len(s1)/10)
start = 0
stop = 200000

In [135]:
sd.play(s1[start:stop]*200, 16000)

In [136]:
sd.play(s2[start:stop]*200, 16000)

In [68]:
sd.play(clean_recon[0:int(len(s1)/20)]*10, 16000)

In [69]:
sd.play(mixed_recon[0:int(len(s1)/20)]*10, 16000)

In [84]:
# data_series = str(4)

In [85]:
# h5f = h5py.File('TIMIT/Organized/concatenated/mixed'  + data_series + '.hdf5','r')
# X = h5f['mixed'  + data_series ][:]
# h5f.close()

In [86]:
# h5f = h5py.File('TIMIT/Organized/concatenated/clean'  + data_series + '.hdf5','r')
# y = h5f['clean'  + data_series ][:]
# h5f.close()

In [89]:
# X = X.T
# y = y.T

In [101]:
# clean_recon1 = reconstruct(y[0:200,fft_samples: fft_samples*2], angle_clean[0:200,fft_samples: fft_samples*2])
# sd.play(clean_recon1*10, 16000)

In [102]:
# mixed_recon1 = reconstruct(X[0:200,0:fft_samples], angle_mixed[0:200])
# sd.play(mixed_recon1*10, 16000)